# Quantum Computing Experimentation with Amazon Braket
### Chapter 5 - Using Quantum Annealer - Developing a QUBO function and using Constraints  
### Party Optimization example
© 2021, 2022, 2023 Packt Publishing

Written by Alex Khan


In [8]:
#run first time
#!pip install pandas

In [9]:
# Set the S3 bucket and folder name
my_bucket = "amazon-braket-pqcee" 
my_prefix = "dwave" 
s3_folder = (my_bucket, my_prefix)

In [19]:
import pandas as pd
import dimod
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
from dwave.system.composites import EmbeddingComposite
from braket.aws import AwsDevice

file_name='friends.csv'

# Read the values into a matrix T
Temp = pd.read_csv(file_name, ).values
T=Temp[:,1:]
print(T)

[[0.5 nan nan nan nan nan]
 [1.0 0.0 nan nan nan nan]
 [0.5 0.25 -0.25 nan nan nan]
 [-0.5 -1.0 0.0 -0.5 nan nan]
 [-1.0 -1.0 0.25 0.5 -1.0 nan]
 [0.25 -0.25 0.0 0.5 -0.5 -0.5]]


In [2]:
dim=len(T[0])
Names=['']*dim

for i in range(dim):
    Names[i]=Temp[i,0]
print(Names)

['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank']


In [3]:
# Notice we change the sign on T
linear={Names[i]:-T[i][i] for i in range(dim)}
quadrati={(Names[i],Names[j]):-T[i][j] for i in range(dim) for j in range(dim) if i>j}

print(linear)

{'Alice': -0.5, 'Bob': -0.0, 'Charlie': 0.25, 'David': 0.5, 'Eve': 1.0, 'Frank': 0.5}


In [25]:
print(quadratic)

{('Bob', 'Alice'): -1.0, ('Charlie', 'Alice'): -0.5, ('Charlie', 'Bob'): -0.25, ('David', 'Alice'): 0.5, ('David', 'Bob'): 1.0, ('David', 'Charlie'): -0.0, ('Eve', 'Alice'): 1.0, ('Eve', 'Bob'): 1.0, ('Eve', 'Charlie'): -0.25, ('Eve', 'David'): -0.5, ('Frank', 'Alice'): -0.25, ('Frank', 'Bob'): 0.25, ('Frank', 'Charlie'): -0.0, ('Frank', 'David'): -0.5, ('Frank', 'Eve'): 0.5}


In [26]:
def estimate_cost(device,num_shots=1000):
    #device=set_device(Name)
    cost_per_task=0.30
    Name=device.name
    if Name in ['SV1','TN1','dm1']:
        price_per_min=device.properties.service.deviceCost.price
        unit=device.properties.service.deviceCost.unit
        print('simulator cost per ',unit,': $', price_per_min)
        print('total cost cannot be estimated')
    elif Name in['Aspen-M-3', 'Aquila', 'Lucy']:
        price_per_shot=device.properties.service.deviceCost.price
        unit=device.properties.service.deviceCost.unit
        print('device cost per ',unit,': $', price_per_shot)
        print('total cost for {} shots is ${:.2f}'.format(num_shots,cost_per_task+num_shots*price_per_shot))
    else:
        print('device not found')
        print('use name from list', available_devices())

In [16]:
def estimate_cost_measured_qubits(device,measured_qubits):
    device=set_device(Name)
    min_shots_per_variable=25
    max_shots=device.properties.service.shotsRange[1]
    print('max shots:', max_shots)
    num_shots=min_shots_per_variable*2**measured_qubits
    if num_shots>max_shots:
        num_shots=max_shots
        print('for {} measured qubits the maximum allowed shots: {:,}'.format(measured_qubits,num_shots))
    else:
        print('for {} measured qubits the number of shots recommended: {:,}'.format(measured_qubits,num_shots))
    estimate_cost(device,num_shots)

In [21]:
def set_device(Name):
    device_list=AwsDevice.get_devices(names=Name)
    if len(device_list)==1:
        device=device_list[0]
        print(device)
        return(device)
    else:
        print('No device found')
        print('use name from list', available_devices()) 

In [22]:
def available_devices():
    from braket.aws import AwsDevice
    device_list=AwsDevice.get_devices(statuses=['ONLINE'])
    device_name_list=[]
    for device in device_list:
        device_name_list.append(device.name)
    #print('Valid device names: ',device_name_list)
    return(device_name_list)

In [40]:
device=set_device('SV1')
estimate_cost_measured_qubits(device, 5)

Device('name': SV1, 'arn': arn:aws:braket:::device/quantum-simulator/amazon/sv1)
max shots: 100000
for 5 measured qubits the number of shots recommended: 800
simulator cost per  minute : $ 0.075
total cost cannot be estimated


In [43]:
vartype = dimod.BINARY
bqm = dimod.BinaryQuadraticModel(linear, quadratic, vartype)

# Using ExactSolver()
# sampler = dimod.ExactSolver()

# Using Rigetti Annealer 
sampler = BraketDWaveSampler(s3_folder)
# sampler = EmbeddingComposite(sampler) 

response = sampler.sample(bqm)
print(response)

BinaryQuadraticModelStructureError: Problem graph incompatible with solver.